Notebook 02 - Análisis de Sentimiento

Objetivo: Limpieza final + análisis de sentimiento de comentarios del CCD


Importación de librerías y carga del csv. 

In [3]:
import pandas as pd         
from textblob import TextBlob  
import numpy as np            
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')  
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re


[nltk_data] Downloading package vader_lexicon to C:\Users\Juan
[nltk_data]     Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
#Importar el csv
df = pd.read_csv('../data/processed/comentarios_limpio.csv')

Obtención de sentimientos. 

In [5]:
#Análisis de sentimiento con TextBlob
def analizar_sentimiento(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity, blob.sentiment.subjectivity

"""
La polaridad indica si el sentimiento del texto es positivo, negativo o neutral.
Su valor va de -1.0 a 1.0
-1.0 → Sentimiento muy negativo
0 → Sentimiento neutral
1.0 → Sentimiento muy positivo

La subjetividad mide qué tan opinativo o personal es el texto.
Su valor va de 0.0 a 1.0
0.0 → Texto muy objetivo (basado en hechos)
1.0 → Texto muy subjetivo (basado en opiniones, emociones, percepciones)
"""


df[['polarity', 'subjectivity']] = df['comentario'].apply(lambda x: pd.Series(analizar_sentimiento(x))).round(2)

In [6]:
df.head(100)

,id,usuario,comentario,fecha,fuente,polarity,subjectivity
0,1,ana,does not apply,2025-01-01,sin fuente,0.00,0.00
1,2,ana,"it doesn't work, i am very disappointed",2025-01-02,app,-0.98,0.98
2,3,rocio,very good experience,2025-01-02,encuesta,0.91,0.78
3,4,martin,very good experience,2025-01-04,encuesta,0.91,0.78
4,5,sofia,"i loved the service, everything very fast",2025-01-04,encuesta,0.48,0.79
...,...,...,...,...,...,...,...
94,96,sofia,"it doesn't work, i am very disappointed",2025-04-06,webe,-0.98,0.98
95,97,lucas,"i loved the service, everything very fast",2025-04-07,app,0.48,0.79
96,98,carla,very good experience,2025-04-08,web,0.91,0.78
97,99,maria,the product never arrived,2025-04-09,web,0.00,0.00


## Análisis de sentimiento con TextBlob

El análisis de sentimiento aplicado a los comentarios permite identificar dos dimensiones clave: **polaridad** y **subjetividad**. La polaridad muestra si el comentario expresa una valoración positiva, negativa o neutral; mientras que la subjetividad indica el grado en que el texto contiene opiniones o expresiones personales.

## Resultados generales

Al aplicar la función de análisis con TextBlob, se observa que los valores obtenidos reflejan con claridad los tonos emocionales presentes en los textos:

- **Comentarios negativos:** frases como *"it doesn't work, I am very disappointed"* presentan una polaridad muy baja (-0.98) acompañada de una subjetividad alta (0.98). Esto indica una opinión personal clara y cargada de insatisfacción.
  
- **Comentarios positivos:** expresiones como *"very good experience"* o *"I loved the service, everything very fast"* muestran polaridades positivas (0.48 a 0.91) y subjetividades altas (0.78 a 0.79). Aquí predomina la percepción personal favorable.

- **Comentarios neutros u objetivos:** textos como *"does not apply"* o *"the product never arrived"* registran polaridad 0.00 y subjetividad 0.00. Esto señala mensajes informativos sin carga emocional.

## Observaciones destacadas

- Existe un contraste marcado entre los comentarios altamente subjetivos, que suelen ser los más extremos (tanto positivos como negativos), y aquellos completamente neutros, que reportan hechos sin opinion.
- Los comentarios negativos presentan la mayor intensidad emocional, lo cual es útil para identificar situaciones de insatisfacción crítica.
- Los comentarios positivos muestran consistencia en su valoración, especialmente en frases repetidas dentro del dataset.
- La subjetividad elevada en la mayoría de los comentarios positivos y negativos sugiere que la mayor parte de los usuarios expresan experiencias personales más que descripciones objetivas.

## Conclusión

El análisis confirma que TextBlob es efectivo para distinguir rápidamente patrones emocionales y niveles de opinión dentro de los comentarios. Las métricas obtenidas permiten segmentar los textos en categorías útiles para análisis posteriores, como clasificación por cuadrantes o visualización en dashboards de Kibana.


In [7]:
#Cantidad de comentarios considerados positivos, negativos y neutrales
cantidad_positivos = df[df['polarity'] > 0]
cantidad_negativos = df[df['polarity'] < 0]
cantidad_neutrales = df[df['polarity'] == 0]

print(f"Cantidad de comentarios positivos: {len(cantidad_positivos)}")
print(f"Cantidad de comentarios negativos: {len(cantidad_negativos)}")
print(f"Cantidad de comentarios neutrales: {len(cantidad_neutrales)}")

Cantidad de comentarios positivos: 30
Cantidad de comentarios negativos: 27
Cantidad de comentarios neutrales: 42


## Distribución de comentarios por sentimiento

La clasificación de los comentarios según su polaridad muestra un panorama equilibrado entre opiniones positivas y negativas, pero con una presencia considerable de mensajes neutrales.

### Resultados principales

- **Comentarios positivos:** 30  
  Representan textos con una valoración satisfactoria o experiencias favorables. Suelen presentar polaridades superiores a 0.2, acompañadas de subjetividades altas debido al carácter personal de las opiniones.

- **Comentarios negativos:** 27  
  Incluyen reclamos, quejas o insatisfacción marcada. Los valores de polaridad cercanos a -1 reflejan emociones intensas, lo que permite identificar casos críticos dentro del dataset.

- **Comentarios neutrales:** 42  
  Constituyen el grupo más numeroso. Este tipo de comentarios no expresa emociones claras y tiende a contener frases informativas o de descripción objetiva.

## Conclusiones

- La mayor parte de los comentarios corresponden a **mensajes neutrales**, lo que indica que muchos usuarios se limitan a describir hechos sin manifestar emociones.
- Los comentarios negativos muestran mayor intensidad emocional, lo que los convierte en insumos valiosos para identificar oportunidades de mejora.
- Los comentarios positivos están presentes en buena proporción y suelen repetirse en distintas fuentes, señalando experiencias consistentes.
- La distribución por cuadrantes evidencia una ligera predominancia de opiniones positivas subjetivas frente a las negativas subjetivas, aunque la diferencia no es amplia.


In [8]:


# Crear columna con cuadrante
def asignar_cuadrante(row):
    p = row['polarity']
    s = row['subjectivity']
    
    if p > 0 and s > 0.5:
        return 'Q1 (Positivo - Subjetivo)'
    elif p < 0 and s > 0.5:
        return 'Q2 (Negativo - Subjetivo)'
    elif p < 0 and s <= 0.5:
        return 'Q3 (Negativo - Objetivo)'
    elif p > 0 and s <= 0.5:
        return 'Q4 (Positivo - Objetivo)'
    else:
        return 'Sin cuadrante (polaridad = 0)'  # neutrales

df['cuadrante'] = df.apply(asignar_cuadrante, axis=1)

# Conteo por cuadrante
conteo_cuadrantes = df['cuadrante'].value_counts()

print(conteo_cuadrantes)


cuadrante
Sin cuadrante (polaridad = 0)    42
Q1 (Positivo - Subjetivo)        22
Q2 (Negativo - Subjetivo)        19
Q4 (Positivo - Objetivo)          8
Q3 (Negativo - Objetivo)          8
Name: count, dtype: int64


## Clasificación de comentarios por cuadrantes

La asignación de cuadrantes combina dos dimensiones del análisis de sentimientos: **polaridad** (positiva, negativa o neutra) y **subjetividad** (opinión personal vs. mensaje objetivo). Esta clasificación permite identificar no solo el tono del comentario, sino también la forma en que está expresado.

## Resultados del conteo por cuadrante

- **Sin cuadrante (polaridad = 0): 42**  
  Corresponde a los comentarios neutrales. Estos textos no expresan emociones claras y suelen limitarse a describir situaciones o hechos sin valoración.

- **Q1 (Positivo – Subjetivo): 22**  
  Comentarios positivos con una carga emocional marcada. Representan experiencias de satisfacción expresadas de manera personal y enfática.

- **Q2 (Negativo – Subjetivo): 19**  
  Opiniones negativas con alto componente emocional. Generalmente incluyen quejas o expresiones de frustración.

- **Q4 (Positivo – Objetivo): 8**  
  Comentarios favorables, pero expresados de forma más neutral y directa, sin énfasis emocional.

- **Q3 (Negativo – Objetivo): 8**  
  Comentarios negativos formulados de manera objetiva. Suelen reportar fallas o problemas sin un tono emocional explícito.

## Conclusiones

- La mayor proporción de comentarios se concentra en el grupo **neutral**, lo que indica una predominancia de textos descriptivos sobre opiniones emocionales.
- Los cuadrantes **subjetivos** (Q1 y Q2) reúnen una parte importante del total, mostrando que tanto las experiencias positivas como las negativas tienden a expresarse con carga emocional.
- La presencia equilibrada entre los cuadrantes positivos y negativos (tanto subjetivos como objetivos) sugiere un nivel de diversidad en las experiencias reportadas.
- Este tipo de segmentación será útil para el dashboard de Kibana, ya que permite visualizar no solo el sentimiento, sino también la intensidad y el estilo de expresión del usuario.


In [10]:
df.to_csv("../data/processed/sentiment_analysis_pbi.csv", index=False)